In [1]:
import sympy
import sympy.physics.mechanics as mech
import control
import pylab as pl
import pickle
sympy.init_printing(use_latex='mathjax')
mech.init_vprinting(use_latex='mathjax')
%matplotlib inline

In [2]:
t, m, Jx, Jy, Jz, Jxz, P, Q, R, U, V, W, phi, theta, psi, p_N, p_E, p_D = \
    sympy.symbols('t m Jx Jy Jz Jxz P Q R U V W phi theta psi p_N p_E p_D')
F_bx, F_by, F_bz, M_bx, M_by, M_bz = sympy.symbols('F_bx F_by F_bz M_bx M_by M_bz')

In [3]:
frame_i = mech.ReferenceFrame('i')
frame_b = frame_i.orientnew('b', 'Body', (psi(t), theta(t), phi(t)), '321')
omega_ib = P(t)*frame_b.x + Q(t)*frame_b.y + R(t)*frame_b.z
sol_euler_rates = sympy.solve((frame_b.ang_vel_in(frame_i) - omega_ib).to_matrix(frame_b),
            [xi(t).diff(t) for xi in [phi, theta, psi]])
frame_b.set_ang_vel(frame_i, omega_ib)
sol_euler_rates

⎧                                               Q⋅sin(φ) + R⋅cos(φ)           
⎨φ̇: P + Q⋅sin(φ)⋅tan(θ) + R⋅cos(φ)⋅tan(θ), ψ̇: ───────────────────, θ̇: Q⋅cos
⎩                                                      cos(θ)                 

              ⎫
(φ) - R⋅sin(φ)⎬
              ⎭

In [4]:
point_o = mech.Point('o')
point_o.set_vel(frame_i, 0)
point_cm = point_o.locatenew('cm', p_N(t)*frame_i.x + p_E(t)*frame_i.y + p_D(t)*frame_i.z)
point_cm.set_vel(frame_b, 0)
point_cm.set_vel(frame_i, point_cm.pos_from(point_o).diff(t, frame_i))
V_i = U(t)*frame_b.x + V(t)*frame_b.y + W(t)*frame_b.z
sol_kin_trans = sympy.solve((V_i - point_cm.vel(frame_i)).to_matrix(frame_i), [xi(t).diff(t) for xi in [p_N, p_E, p_D]])
point_cm.set_vel(frame_i, V_i)
sol_kin_trans

{p_̇D: -U⋅sin(θ) + V⋅sin(φ)⋅cos(θ) + W⋅cos(φ)⋅cos(θ), p_̇E: U⋅sin(ψ)⋅cos(θ) + 
V⋅sin(φ)⋅sin(ψ)⋅sin(θ) + V⋅cos(φ)⋅cos(ψ) - W⋅sin(φ)⋅cos(ψ) + W⋅sin(ψ)⋅sin(θ)⋅c
os(φ), p_̇N: U⋅cos(ψ)⋅cos(θ) + V⋅sin(φ)⋅sin(θ)⋅cos(ψ) - V⋅sin(ψ)⋅cos(φ) + W⋅si
n(φ)⋅sin(ψ) + W⋅sin(θ)⋅cos(φ)⋅cos(ψ)}

In [5]:
J = mech.inertia(frame_b, ixx=Jx, iyy=Jy, izz=Jz, izx=Jxz)
body = mech.RigidBody('body', point_cm, frame_b, m, (J, point_cm))
def bke(vect, frame_i, frame_b, t):
    return vect.diff(t, frame_b) + frame_b.ang_vel_in(frame_i).cross(vect)

In [6]:
F_b = F_bx(t)*frame_b.x +  F_by(t)*frame_b.y + F_bz(t)*frame_b.z
sol_trans = sympy.solve((bke(body.linear_momentum(frame_i), frame_i, frame_b, t) - F_b).to_matrix(frame_b),
            [xi(t).diff(t) for xi in [U, V, W]])
sol_trans

⎧                 F_bx                  F_by                   F_bz⎫
⎨U̇: -Q⋅W + R⋅V + ────, V̇: P⋅W - R⋅U + ────, Ẇ: -P⋅V + Q⋅U + ────⎬
⎩                  m                     m                      m  ⎭

In [7]:
M_b = M_bx(t)*frame_b.x +  M_by(t)*frame_b.y + M_bz(t)*frame_b.z
sol_rot = sympy.solve((bke(body.angular_momentum(point_cm, frame_i), frame_i, frame_b, t) - M_b).to_matrix(frame_b),
                      [xi(t).diff(t) for xi in [P, Q, R]])
sol_rot

⎧                                                                             
⎪    -(Jxz⋅(Jx⋅P⋅Q + Jxz⋅Q⋅R - Jy⋅P⋅Q + M_bz) + Jz⋅(Jxz⋅P⋅Q - Jy⋅Q⋅R + Jz⋅Q⋅R 
⎨Ṗ: ─────────────────────────────────────────────────────────────────────────
⎪                                                  2                          
⎩                                       Jx⋅Jz - Jxz                           

                              2        2                                      
- M_bx))       -Jx⋅P⋅R + Jxz⋅P  - Jxz⋅R  + Jz⋅P⋅R + M_by      Jx⋅(Jx⋅P⋅Q + Jxz
─────────, Q̇: ─────────────────────────────────────────, Ṙ: ────────────────
                                   Jy                                         
                                                                              

                                                              ⎫
⋅Q⋅R - Jy⋅P⋅Q + M_bz) + Jxz⋅(Jxz⋅P⋅Q - Jy⋅Q⋅R + Jz⋅Q⋅R - M_bx)⎪
──────────────────────────────────────────────────────────────⎬
                

In [8]:
eoms = {}
eoms.update(sol_euler_rates)
eoms.update(sol_trans)
eoms.update(sol_rot)
eoms.update(sol_kin_trans)
x_vect = sympy.Matrix([U, V, W, P, Q, R, phi, theta, psi, p_N, p_E, p_D])
u_vect = sympy.Matrix([F_bx, F_by, F_bz, M_bx, M_by, M_bz])
sub_no_t = {xi(t):xi for xi in list(x_vect) + list(u_vect)}
f_vect = sympy.Matrix([eoms[xi(t).diff(t)].subs(sub_no_t) for xi in x_vect])
g_vect = x_vect
x_vect.T, u_vect.T

([U  V  W  P  Q  R  φ  θ  ψ  p_N  p_E  p_D], [F_bx  F_by  F_bz  M_bx  M_by  M_
bz])

In [9]:
f_vect

⎡                                          F_bx                               
⎢                                          ──── - Q⋅W + R⋅V                   
⎢                                           m                                 
⎢                                                                             
⎢                                          F_by                               
⎢                                          ──── + P⋅W - R⋅U                   
⎢                                           m                                 
⎢                                                                             
⎢                                          F_bz                               
⎢                                          ──── - P⋅V + Q⋅U                   
⎢                                           m                                 
⎢                                                                             
⎢         -(Jxz⋅(Jx⋅P⋅Q + Jxz⋅Q⋅R - Jy⋅P⋅Q + M_bz) +

In [10]:
#A = f_vect.jacobian(x_vect).applyfunc(lambda e: e.simplify())
#B = f_vect.jacobian(u_vect).applyfunc(lambda e: e.simplify())
#C = g_vect.jacobian(x_vect).applyfunc(lambda e: e.simplify())
#D = g_vect.jacobian(u_vect).applyfunc(lambda e: e.simplify())
#sub_const = {
#    m:1,
#    Jx:1,
#    Jy:1,
#    Jz:1,
#    Jxz:0,
#}
#sub_x0 = {
#    phi:0,
#    theta:0,
#    psi:0,
#    P:0,
#    Q:0,
#    R:0,
#    U:0,
#    V:0,
#    W:0,
#}
#ss0 = control.ss(*[pl.array(xi.subs(sub_x0).subs(sub_const)).astype(float) for xi in [A, B, C, D]])
#control.bode(ss0[0,0]);
#control.rlocus(ss0[0,0]);